In [63]:
# import all relevant libraries
import pyodbc # to access Ms Access database

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

In [64]:
# connect to Ms Access database

db_path = r"C:\Users\GILBERT FG\Desktop\Readings\PSP_database.accdb" # path to Ms access database

conn = pyodbc.connect(
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    rf"DBQ={db_path};"
)

cursor = conn.cursor()

# list all tables
for table in cursor.tables(tableType="TABLE"):
    print(table.table_name)


Block register
Plot monitoring history
Plot-block relation
PSPs
Slope correction factors
Thinning history
TreeData


In [65]:
# Queries
# Select all from Tree Data and filter the teak trees from Tain II forest reserve
tree_data_query = r"SELECT * FROM TreeData WHERE AreaType = 'Teak' AND Plantations = 'Tain II'"

# read the dataset into dataframe
df = pd.read_sql(tree_data_query, conn)

# preview the dataset
df.head()

C:\Users\GILBERT FG\AppData\Local\Temp\ipykernel_13648\4228675863.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(tree_data_query, conn)


,Plantations,AreaType,Monitoring year,Monitoring month,Monitoring day,PLOT,TREE NR,Tree SPECIES,Species scientific name,H (m),DBH (cm),Merchantable height (m),REMARKS,Incorrect DBH,Incorrect H,Incorrect H / DBH,Exclude,Justification for exclusion
0,Tain II,Teak,2018.0,1.0,13,1,71.0,Teak,Tectona grandis,7.25,12.0,None,Fire scars,False,False,None,False,None
1,Tain II,Teak,2018.0,1.0,13,1,76.0,Teak,Tectona grandis,8.25,12.0,None,Fire scars,False,False,None,False,None
2,Tain II,Teak,2018.0,1.0,13,1,68.0,Teak,Tectona grandis,7.00,11.8,None,Fire scars,False,False,None,False,None
3,Tain II,Teak,2018.0,1.0,13,1,53.0,Teak,Tectona grandis,7.25,11.5,None,Fire scars,False,False,None,False,None
4,Tain II,Teak,2018.0,1.0,13,1,58.0,Teak,Tectona grandis,7.00,10.5,None,Fire scars,False,False,None,False,None


In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191860 entries, 0 to 191859
Data columns (total 18 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Plantations                  191860 non-null  object 
 1   AreaType                     191860 non-null  object 
 2   Monitoring year              191860 non-null  float64
 3   Monitoring month             191860 non-null  float64
 4   Monitoring day               191860 non-null  int64  
 5   PLOT                         191860 non-null  object 
 6   TREE NR                      191860 non-null  float64
 7   Tree SPECIES                 191860 non-null  object 
 8   Species scientific name      191851 non-null  object 
 9   H (m)                        191804 non-null  float64
 10  DBH (cm)                     191800 non-null  float64
 11  Merchantable height (m)      0 non-null       object 
 12  REMARKS                      70254 non-null   object 
 13 

In [67]:
df['Monitoring year'] = df['Monitoring year'].astype(int)
df['Monitoring month'] = df['Monitoring month'].astype(int)

In [68]:
df['Monitoring year'] = df['Monitoring year'].astype(str)
df['Monitoring month'] = df['Monitoring month'].astype(str)
df['Monitoring day'] = df['Monitoring day'].astype(str)

In [74]:
df['Monitoring date'] = df['Monitoring day'] + '/'+ df['Monitoring month'] + '/' + df['Monitoring year']
df.head()

,Plantations,AreaType,Monitoring year,Monitoring month,Monitoring day,PLOT,TREE NR,Tree SPECIES,Species scientific name,H (m),DBH (cm),Merchantable height (m),REMARKS,Incorrect DBH,Incorrect H,Incorrect H / DBH,Exclude,Justification for exclusion,Monitoring date
0,Tain II,Teak,2018,1,13,1,71.0,Teak,Tectona grandis,7.25,12.0,None,Fire scars,False,False,None,False,None,13/1/2018
1,Tain II,Teak,2018,1,13,1,76.0,Teak,Tectona grandis,8.25,12.0,None,Fire scars,False,False,None,False,None,13/1/2018
2,Tain II,Teak,2018,1,13,1,68.0,Teak,Tectona grandis,7.00,11.8,None,Fire scars,False,False,None,False,None,13/1/2018
3,Tain II,Teak,2018,1,13,1,53.0,Teak,Tectona grandis,7.25,11.5,None,Fire scars,False,False,None,False,None,13/1/2018
4,Tain II,Teak,2018,1,13,1,58.0,Teak,Tectona grandis,7.00,10.5,None,Fire scars,False,False,None,False,None,13/1/2018


In [75]:
# trim the dataset into relevant features
new_dataset =df[['Plantations', 'AreaType', 'Monitoring date', 'PLOT', 'TREE NR', 'Tree SPECIES', 'H (m)', 'DBH (cm)']]

new_dataset.head()

,Plantations,AreaType,Monitoring date,PLOT,TREE NR,Tree SPECIES,H (m),DBH (cm)
0,Tain II,Teak,13/1/2018,1,71.0,Teak,7.25,12.0
1,Tain II,Teak,13/1/2018,1,76.0,Teak,8.25,12.0
2,Tain II,Teak,13/1/2018,1,68.0,Teak,7.00,11.8
3,Tain II,Teak,13/1/2018,1,53.0,Teak,7.25,11.5
4,Tain II,Teak,13/1/2018,1,58.0,Teak,7.00,10.5


In [76]:
new_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191860 entries, 0 to 191859
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Plantations      191860 non-null  object 
 1   AreaType         191860 non-null  object 
 2   Monitoring date  191860 non-null  object 
 3   PLOT             191860 non-null  object 
 4   TREE NR          191860 non-null  float64
 5   Tree SPECIES     191860 non-null  object 
 6   H (m)            191804 non-null  float64
 7   DBH (cm)         191800 non-null  float64
dtypes: float64(3), object(5)
memory usage: 11.7+ MB


In [77]:
# create consistent headers (features) especially, the headers should have the same font type
new_dataset.rename(columns={'PLOT': 'Plot no.', 'TREE NR': 'Tree no.', 'Tree SPECIES': 'Tree species', 'AreaType': 'Area type'})

,Plantations,Area type,Monitoring date,Plot no.,Tree no.,Tree species,H (m),DBH (cm)
0,Tain II,Teak,13/1/2018,1,71.0,Teak,7.25,12.0
1,Tain II,Teak,13/1/2018,1,76.0,Teak,8.25,12.0
2,Tain II,Teak,13/1/2018,1,68.0,Teak,7.00,11.8
3,Tain II,Teak,13/1/2018,1,53.0,Teak,7.25,11.5
4,Tain II,Teak,13/1/2018,1,58.0,Teak,7.00,10.5
...,...,...,...,...,...,...,...,...
191855,Tain II,Teak,13/3/2024,1421,76.0,Teak,0.45,0.0
191856,Tain II,Teak,13/3/2024,1421,77.0,Teak,0.75,0.0
191857,Tain II,Teak,13/3/2024,1421,78.0,Teak,1.75,2.0
191858,Tain II,Teak,13/3/2024,1421,80.0,Teak,0.90,0.0


In [78]:
new_dataset['Monitoring date'] = pd.to_datetime(new_dataset['Monitoring date'])

C:\Users\GILBERT FG\AppData\Local\Temp\ipykernel_13648\1481374675.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  new_dataset['Monitoring date'] = pd.to_datetime(new_dataset['Monitoring date'])
C:\Users\GILBERT FG\AppData\Local\Temp\ipykernel_13648\1481374675.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dataset['Monitoring date'] = pd.to_datetime(new_dataset['Monitoring date'])


In [79]:
new_dataset.head()

,Plantations,AreaType,Monitoring date,PLOT,TREE NR,Tree SPECIES,H (m),DBH (cm)
0,Tain II,Teak,2018-01-13,1,71.0,Teak,7.25,12.0
1,Tain II,Teak,2018-01-13,1,76.0,Teak,8.25,12.0
2,Tain II,Teak,2018-01-13,1,68.0,Teak,7.00,11.8
3,Tain II,Teak,2018-01-13,1,53.0,Teak,7.25,11.5
4,Tain II,Teak,2018-01-13,1,58.0,Teak,7.00,10.5


In [80]:
# sort records for each tree over time
sorted_dataset = new_dataset.sort_values(by=['Plot no.', 'Tree no.', 'Monitoring date'])

KeyError: 'Plot no.'

In [21]:
sorted_dataset.head()

,Plantations,Area type,Monitoring year,Plot no.,Tree no.,Tree species,H (m),DBH (cm)
16,Tain II,Teak,2018.0,1,1.0,Teak,5.00,6.1
59,Tain II,Teak,2019.0,1,1.0,Teak,7.50,8.9
67638,Tain II,Teak,2020.0,1,1.0,Teak,9.75,12.5
111343,Tain II,Teak,2021.0,1,1.0,Teak,12.50,16.0
160935,Tain II,Teak,2022.0,1,1.0,Teak,13.25,18.3


### Calculate Tree Growth

##### Diameter annual increment

In [22]:
sorted_dataset['DBH_lag'] = sorted_dataset.groupby('Tree no.')['DBH (cm)'].shift(1)
sorted_dataset['Year_lag'] = sorted_dataset.groupby('Tree no.')['Monitoring year'].shift(1)
sorted_dataset['Years'] = (sorted_dataset['Monitoring year'] - sorted_dataset['Year_lag']).dt.days / 365.25


AttributeError: Can only use .dt accessor with datetimelike values